- first modelling, no FE
- mix stratified kold and kfold
- nfolds 4
- only check oof score
- do not label encode cp_time

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm._tqdm_notebook import tqdm_notebook

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  import sys


In [2]:
DIR = "/kaggle/input/lish-moa/"
train_feat = pd.read_csv(DIR+"train_features.csv")
test_feat = pd.read_csv(DIR+"test_features.csv")
#train_nonscore = pd.read_csv(DIR+"train_targets_nonscored.csv")
train_score = pd.read_csv(DIR+"train_targets_scored.csv")
sub = pd.read_csv(DIR+"sample_submission.csv")

In [3]:
target_feats = [ i for i in train_score.columns if i != "sig_id"]
g_feats = [i for i in train_feat.columns if "g-" in i]
c_feats = [i for i in train_feat.columns if "c-" in i]

In [4]:
noncons_train_index = train_feat[train_feat.cp_type=="ctl_vehicle"].index
noncons_test_index = test_feat[test_feat.cp_type=="ctl_vehicle"].index

In [5]:
def g_calc(row):
    return len(np.unique(row[g_feats].values))

def c_calc(row):
    return len(np.unique(row[c_feats].values))

categoricals = ["cp_type", "cp_dose"]

tqdm_notebook.pandas(desc="progress")
train_feat["g_unique_num"] = train_feat.progress_apply(g_calc, axis=1)
test_feat["g_unique_num"] = test_feat.progress_apply(g_calc, axis=1)

for f in categoricals:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_feat[f])+[-999])
    train_feat[f] = lbl.transform(list(train_feat[f]))
    test_feat[f] = lbl.transform(list(test_feat[f])) 

#train_feat["c_unique_num"] = train_feat.progress_apply(c_calc, axis=1)
#test_feat["c_unique_num"] = test_feat.progress_apply(c_calc, axis=1)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
lgbm_params = {'objective': 'binary', 'metric': 'binary_logloss', 'eval_metric': 'logloss', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.01, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def modelling_lgb(new_train, target_train, new_test, target):
    
    X_train = new_train.drop(['sig_id'],axis=1).copy()
    y_train = target_train[target].copy()
    X_test = new_test.copy()
    X_test = new_test.drop(['sig_id'],axis=1).copy()
        
    pred_value = np.zeros(X_test.shape[0])

    #remove_features = []
    #for i in X_train.columns:
    #    if (X_train[i].std() == 0) and i not in remove_features:
    #        remove_features.append(i)
    #X_train = X_train.drop(remove_features, axis=1)
    #X_test = X_test.drop(remove_features, axis=1)
    
    features_list = [i for i in X_train.columns]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    
    n_folds=4
    if target not in ["erbb2_inhibitor", "atp-sensitive_potassium_channel_antagonist"]:
        skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    else:
        skf=KFold(n_splits = n_folds, shuffle=True, random_state=0)

    models = []

    valid = np.zeros([X_train.shape[0]])
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_valid2 = X_train.iloc[test_index,:]
        y_valid2 = y_train.iloc[test_index]
        
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_valid2, y_valid2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
               num_boost_round=10000,early_stopping_rounds=50,verbose_eval = 1000) 

        valid_predict = clf.predict(X_valid2, num_iteration = clf.best_iteration)
        valid[test_index] = valid_predict
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
    feature_importance_df = feature_importance_df.sort_values("Average", ascending=False).reset_index(drop=True)

    score = log_loss(y_train, valid)
    print("oof log_loss= {} ".format(score))
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    
    return valid, pred_value, feature_importance_df,score

train_checkscore = train_score.copy()
difficult_list = pd.DataFrame(target_feats, columns=["Target"])
difficult_list["Oof log_loss"] = 0
difficult_list.set_index("Target")

for ind, target in enumerate(target_feats):
    print(ind, target)
    valid, pred_value, feature_importance_df, score = modelling_lgb(train_feat, train_score, test_feat, target)
    train_checkscore[target] = valid
    difficult_list.loc[target, "Oof log_loss"] = score
    print("top10: ", list(feature_importance_df.iloc[:10]["Feature"]))
    print("last10: ", list(feature_importance_df.iloc[-10:]["Feature"]))
    del feature_importance_df
    sub[target] = pred_value

0 5-alpha_reductase_inhibitor
Fold 1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00521057	valid_1's binary_logloss: 0.00559618
Fold 2
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[111]	training's binary_logloss: 0.000619286	valid_1's binary_logloss: 0.00662023
Fold 3
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[89]	training's binary_logloss: 0.000820992	valid_1's binary_logloss: 0.00495131
Fold 4
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's binary_logloss: 0.00527654	valid_1's binary_logloss: 0.00559409
oof log_loss= 0.005690487288124951 
top10:  ['g-211', 'c-61', 'g-278', 'g-679', 'g-235', 'g-79', 'g-364', 'g-134', 'g-351', 'g-403']
last10:  ['g-354', 'g-338', 'g-340', 'g-341', 'g-345', 'g-346', 'g-349', 'g-350', 'g-352', 'g_unique_num

In [7]:
# local score
scores = []
for target_col in target_feats:
    scores.append(log_loss(train_checkscore[target_col], train_score[target_col]))
print(np.mean(scores))

ValueError: Unknown label type: (0        0.000303
1        0.000722
2        0.000722
3        0.000301
4        0.000224
           ...   
23809    0.001099
23810    0.000722
23811    0.000331
23812    0.000223
23813    0.000722
Name: 5-alpha_reductase_inhibitor, Length: 23814, dtype: float64,)

In [8]:
np.mean(difficult_list["Oof log_loss"])

0.008027556401360747

In [9]:
np.transpose(difficult_list)

,0,1,2,3,4,5,6,7,8,9,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
Target,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oof log_loss,0,0,0,0,0,0,0,0,0,0,...,0.00234569,0.00730901,0.0143777,0.0239761,0.0202788,0.00239603,0.0266993,0.00845531,0.00819305,0.00964091


In [10]:
sub.to_csv("submission.csv")